In [48]:
from transformers import BertTokenizer,BertModel,BertForSequenceClassification, AdamW, BertConfig, PreTrainedTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn


## Loading Data

In [4]:
df=pd.read_csv('../raw_data/starter_title_train.csv')

In [5]:
df.head()

,Unnamed: 0,id,channel_id,title,views
0,0,xL0ch83RAK8,UC8ROUUjHzEQm-ndb69CX8Ww,台視新聞台HD 24 小時線上直播｜TAIWAN TTV NEWS HD (Live)｜台湾...,106950818
1,0,JOlc_ksXncM,UCY64_tffZ71RniyKDj5bWRA,Crispy French Fries Recipe! You Can Make PERFE...,1340138
2,0,doqBgKIBBh8,UCUaLy1_4rsLo4HyCLCPlD4g,🔴 LIVE WEBCAM from LANZAROTE AIRPORT,18880210
3,0,PM4JQE-35O4,UCDfYcuNw-ff952Xn0VDWwDw,SO WE GOT BANDICOOT-NAPPED... || Crash Bandico...,90917
4,0,gBifepJRAXU,UCDfYcuNw-ff952Xn0VDWwDw,N.GIN HAS MADE HIS OWN ANIMATRONIC? || Crash B...,101879


## Preprocessing

In [6]:
def preprocessing(sentence):
    #stripping emojis
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)
    sentence=deEmojify(sentence)
    
    #stripping punctuations
    for char in string.punctuation:
        sentence=sentence.replace(char,'')
        
    #stripping chinese punctuations
    sentence=re.sub(u'(?<=[^0-9])[^\u4e00-\u9fff0-9a-zA-Z]+(?=[^0-9])',' ',sentence)
    
    #turning lowercase
    sentence=''.join(re.findall(r'\D',(sentence.strip()).lower()))
    
    #tokenizing words
    sentence=word_tokenize(sentence)
    
    #lemmatizing words
    sentence=[WordNetLemmatizer().lemmatize(w) for w in sentence]

    
    return sentence



In [50]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [39]:
# tokenized_df = df['title'].apply(lambda x:tokenizer.tokenize(x))

In [40]:
input_ids = df['title'].apply(lambda x:tokenizer.encode(x, add_special_tokens=True))

In [41]:
max_length=input_ids.apply(lambda x:len(x)).max()


In [47]:
! pip freeze | grep transfo

transformers==2.2.0


In [52]:
ec=tokenizer.tokenize(
            text=df.title.tolist(),# Sentence to encode.
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = max_length,         # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',     # Return pytorch tensors.
            truncation=True)


TypeError: unhashable type: 'list'

In [46]:
input_ids = ec['input_ids']
attention_mask = ec['attention_mask']

NameError: name 'ec' is not defined

In [13]:
input_ids[0]

NameError: name 'input_ids' is not defined

In [14]:
# def encode_sentence(sentence):
#     encoded_corpus=tokenizer(
#                         sentence,                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                         max_length = max_length,           # Pad & truncate all sentences.
#                         pad_to_max_length = True,
#                         return_attention_mask = True,   # Construct attn. masks.
#                         return_tensors = 'pt',     # Return pytorch tensors.
#                         truncation=True)
#     input_ids = encoded_corpus['input_ids']
#     attention_mask = encoded_corpus['attention_mask']
#     return input_ids,attention_mask

In [15]:
# input_ids=df['title'].apply(lambda x:encode_sentence(x)[0]).tolist()
# attention_mask=df['title'].apply(lambda x:encode_sentence(x)[1]).tolist()
labels=np.array(df['views'].tolist())

In [16]:
labels.shape

(744,)

In [17]:
from sklearn.model_selection import train_test_split

test_size = 0.1
seed = 42

train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size, 
                             random_state=seed)

train_masks, test_masks, _, _ = train_test_split(attention_mask, 
                                        labels, test_size=test_size, 
                                        random_state=seed)

NameError: name 'input_ids' is not defined

In [18]:
from sklearn.preprocessing import StandardScaler
price_scaler = StandardScaler()
price_scaler.fit(train_labels.reshape(-1, 1))
train_labels = price_scaler.transform(train_labels.reshape(-1, 1))
test_labels = price_scaler.transform(test_labels.reshape(-1, 1))
#do we need to do this?


NameError: name 'train_labels' is not defined

### Transforming into pytorch friendly input formats (BERT)

In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

def create_dataloaders(inputs, masks, labels, batch_size):
    
    input_tensor = torch.tensor(inputs)
    
    mask_tensor = torch.tensor(masks)
    
    labels_tensor = torch.tensor(labels)
    
    dataset = TensorDataset(input_tensor, mask_tensor, 
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=True)
    return dataloader


train_dataloader = create_dataloaders(train_inputs, train_masks, 
                                      train_labels, batch_size)

test_dataloader = create_dataloaders(test_inputs, test_masks, 
                                     test_labels, batch_size)



NameError: name 'train_inputs' is not defined

In [20]:
train_dataloader

NameError: name 'train_dataloader' is not defined

In [21]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 1,  
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


# Tell pytorch to run this model on the GPU.



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 440473133/440473133 [13:19<00:00, 550782.60B/s]


In [22]:
# import torch
# if torch.cuda.is_available():       
#     device = torch.device("cuda")
#     print("Using GPU.")
# else:
#     print("No GPU available, using the CPU instead.")
#     device = torch.device("cpu")
    
# model.to(device)



In [24]:
#model set to store a double value for regression task
model = model.double()

In [25]:
#optimizer with adam, size and eps
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

In [26]:
#Scheduling training schedule based on number of epochs and step-size
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)

NameError: name 'train_dataloader' is not defined

In [ ]:
#set loss_function as MAE
loss_function = nn.L1Loss()



In [ ]:
params = list(model.named_parameters())

In [ ]:
!pip freeze | grep trans

In [ ]:
!pip install transformers==2.2.0

In [ ]:
import random
import time
from sklearn.metrics import mean_squared_error

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # # ========================================
    # #               Training
    # # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 32 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        
        print(loss)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    y_pred = np.array([])
    y_true = np.array([])

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
 
        # Accumulate the validation loss.
        total_eval_loss += logits.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_pred = np.append(y_pred,logits)
        y_true = np.append(y_true,label_ids)
        
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    print("  RMSE: {0:.4f}".format(rmse))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. RMSE.': rmse,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )


print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))